Here we'll first be loading the FC matrices and explore their structure

In [1]:
#using read_dataset from Datasets/FC/create_dataset.py to read the dataset
from Datasets.FC.create_dataset import read_dataset_MDD
import torch 
print('GPU available?', torch.cuda.is_available())

GPU available? False


In [2]:
dataset = read_dataset_MDD()

In [3]:
#ok, let's explore the data a bit more
#dataset is a list object of torch_geometric.data objects

#let's see the first element
print(dataset[0])

#it's a dictionary object, let's see the keys
print(dataset[0].keys())

#ok, let's see the values
print(dataset[0].values())

#it has 4 keys, 'x', 'edge_index', 'edge_attr' and 'y' where y=0 menas the patient is healthy and y=1 means the patient has MDD
"""graph = Data(x=ROI.reshape(-1,116).float(),
                     edge_index=G.indices().reshape(2,-1).long(),
                     edge_attr=G.values().reshape(-1,1).float(),
                     y=y.long())"""

#how much data do we have?
print('Number of samples:', len(dataset))


Data(x=[116, 116], edge_index=[2, 1294], edge_attr=[1294, 1], y=[1])
['edge_index', 'edge_attr', 'y', 'x']
ValuesView({'x': tensor([[ 0.0000,  0.2857,  0.0804,  ...,  0.2032,  0.1674,  0.0906],
        [ 0.2857,  0.0000, -0.3860,  ...,  0.1637, -0.0359,  0.1674],
        [ 0.0804, -0.3860,  0.0000,  ..., -0.0175, -0.0309,  0.0296],
        ...,
        [ 0.2032,  0.1637, -0.0175,  ...,  0.0000,  0.2329, -0.1452],
        [ 0.1674, -0.0359, -0.0309,  ...,  0.2329,  0.0000,  0.0183],
        [ 0.0906,  0.1674,  0.0296,  ..., -0.1452,  0.0183,  0.0000]]), 'edge_index': tensor([[  0,   0,   0,  ..., 113, 113, 114],
        [ 10,  12,  14,  ..., 111, 112, 108]]), 'edge_attr': tensor([[0.7785],
        [0.6966],
        [0.5463],
        ...,
        [0.6415],
        [0.4745],
        [0.5401]]), 'y': tensor([0])})
Number of samples: 1604


In [4]:
#set the seed
import torch
torch.manual_seed(0)

MODEL = "GCN"
DATASET = "FC_MDD"

from models.models_FC import GCN_framework as framework # import the model

gnn = framework(dataset,device="cpu")

print(gnn.model)
print(gnn.train_idx)

Net(
  (conv_layers): ModuleList(
    (0): GCNConv(116, 128)
    (1-4): 4 x GCNConv(128, 128)
  )
  (batch_norms): ModuleList(
    (0-4): 5 x BatchNorm(128)
  )
  (lin1): Linear(128, 128, bias=True)
  (lin2): Linear(128, 2, bias=True)
  (bn1): BatchNorm(128)
  (bn2): BatchNorm(2)
)
tensor([1365,  984, 1017,  ...,  260,  678,   83])


/home/tpelletreaudur/.local/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [5]:
gnn.iterate()

Epoch: 020, Loss: 0.110, Test Loss: 0.625, Train Acc: 1.000 Test Acc: 0.642
Epoch: 040, Loss: 0.090, Test Loss: 0.614, Train Acc: 1.000 Test Acc: 0.704
Epoch: 060, Loss: 0.077, Test Loss: 0.604, Train Acc: 1.000 Test Acc: 0.704
Epoch: 080, Loss: 0.070, Test Loss: 0.606, Train Acc: 1.000 Test Acc: 0.716
Epoch: 100, Loss: 0.064, Test Loss: 0.604, Train Acc: 1.000 Test Acc: 0.691
Epoch: 120, Loss: 0.062, Test Loss: 0.602, Train Acc: 1.000 Test Acc: 0.704
Epoch: 140, Loss: 0.059, Test Loss: 0.605, Train Acc: 1.000 Test Acc: 0.716


In [6]:
#gnn.train()

In [7]:
#save the model 
gnn.save_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

Model saved in: models/FC_MDD_GCNserver.pt


In [8]:
#save the model 
#gnn.save_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

In [9]:
#load the model
gnn.load_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

In [10]:
gnn.evaluate()

Test Loss: 0.605, Train Acc: 1.000 Test Acc: 0.716


In [11]:
train_features, test_features = gnn.evaluate_with_features2()

In [12]:
print(len(train_features[0]))
len(train_features), len(test_features)

8


(1523, 81)

In [16]:
import networkx as nx
def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Alternative metrics for disconnected graphs
        # Option 1: Use the average path length of the largest connected component
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        density = nx.density(G)
        avg_path_len = calculate_avg_path_length(G)
        num_cliques = len(list(nx.find_cliques(G)))
        num_triangles = sum(nx.triangles(G).values()) / 3
        num_squares = sum(nx.square_clustering(G).values()) / 4
        number_of_node_in_the_largest_fully_connected_component = len(max(nx.connected_components(G), key=len))
        #small_world = nx.algorithms.smallworld.sigma(G)

        properties.append((num_nodes, num_edges, density, avg_path_len, num_cliques, num_triangles, num_squares, number_of_node_in_the_largest_fully_connected_component)) #, small_world))
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties = compute_graph_properties(selected_dataset)

In [17]:
print(len(train_properties))
train_properties[0:5]



1523


[(116,
  488,
  0.07316341829085457,
  3.314842578710645,
  148,
  938.0,
  6.439699892510922,
  116),
 (114,
  598,
  0.0928427262847384,
  3.166744294364229,
  164,
  1378.0,
  7.519630514954544,
  114),
 (114,
  408,
  0.06334420121099209,
  3.391864617295451,
  135,
  511.0,
  5.279502013122494,
  114),
 (115,
  744,
  0.11350114416475973,
  2.796796338672769,
  198,
  1870.0,
  7.980704583203741,
  115),
 (112,
  706,
  0.11357786357786358,
  3.3745173745173744,
  181,
  2528.0,
  9.050100863174576,
  112)]

In [19]:
import pickle as pkl
# save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "wb") as f:
    pkl.dump(train_properties, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "wb") as f:
    pkl.dump(test_properties, f)

#load the properties
with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "rb") as f:
    train_properties = pkl.load(f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "rb") as f:
    test_properties = pkl.load(f)

In [20]:
"""
The embeddings of GCN are like this:
return F.log_softmax(x7, dim=-1), (x1, x2, x3, x4, x5, x_global, x6, x7)
"""


import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
output_size = 1  # Predicting one property at a time
# Define the linear model
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Assume we have already evaluated to get features
#train_features, test_features = gnn.evaluate_with_features2()

# Extract x embeddings
train_x = np.array([feat[0] for feat in train_features])
test_x = np.array([feat[0] for feat in test_features])

# Extract 2, 3, 4, global, 5, 6, 7 embeddings
train_x2 = np.array([feat[1] for feat in train_features])
test_x2 = np.array([feat[1] for feat in test_features])

train_x3 = np.array([feat[2] for feat in train_features])
test_x3 = np.array([feat[2] for feat in test_features])

train_x4 = np.array([feat[3] for feat in train_features])
test_x4 = np.array([feat[3] for feat in test_features])

train_x5 = np.array([feat[4] for feat in train_features])
test_x5 = np.array([feat[4] for feat in test_features])

train_x_global = np.array([feat[5] for feat in train_features])
test_x_global = np.array([feat[5] for feat in test_features])

train_x6 = np.array([feat[6] for feat in train_features])
test_x6 = np.array([feat[6] for feat in test_features])

train_x7 = np.array([feat[7] for feat in train_features])
test_x7 = np.array([feat[7] for feat in test_features])

# Compute graph properties
# train_properties = compute_graph_properties(gnn.dataset[gnn.train_idx])
# test_properties = compute_graph_properties(gnn.dataset[gnn.test_idx])

# Convert to PyTorch tensors
train_x = torch.tensor(train_x, dtype=torch.float32)
train_x2 = torch.tensor(train_x2, dtype=torch.float32)
train_x3 = torch.tensor(train_x3, dtype=torch.float32)
train_x4 = torch.tensor(train_x4, dtype=torch.float32)
train_x5 = torch.tensor(train_x5, dtype=torch.float32)
train_x_global = torch.tensor(train_x_global, dtype=torch.float32)
train_x6 = torch.tensor(train_x6, dtype=torch.float32)
train_x7 = torch.tensor(train_x7, dtype=torch.float32)

test_x = torch.tensor(test_x, dtype=torch.float32)
test_x2 = torch.tensor(test_x2, dtype=torch.float32)
test_x3 = torch.tensor(test_x3, dtype=torch.float32)
test_x4 = torch.tensor(test_x4, dtype=torch.float32)
test_x5 = torch.tensor(test_x5, dtype=torch.float32)
test_x_global = torch.tensor(test_x_global, dtype=torch.float32)
test_x6 = torch.tensor(test_x6, dtype=torch.float32)
test_x7 = torch.tensor(test_x7, dtype=torch.float32)

train_y = torch.tensor(train_properties, dtype=torch.float32)
test_y = torch.tensor(test_properties, dtype=torch.float32)

# Train and evaluate a model for each graph property and each embedding
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings = [(train_x, test_x), (train_x2, test_x2), (train_x3, test_x3), (train_x4, test_x4), (train_x5, test_x5), (train_x_global, test_x_global), (train_x6, test_x6), (train_x7, test_x7)]
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']

In [ ]:
#create a dictionary where we will sotre the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 500000  # Adjust this as needed

        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Adjust this for more frequent/lower print frequency
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y[:, i].cpu().numpy()
            test_target = test_y[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embdedding = embeddings_names[ii]
            results[(name_of_embdedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "wb") as f:
    pkl.dump(results, f)

In [ ]:
# check train_features shape, ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (40,) + inhomogeneous part.
# check train_features[0] shape, (40,) -> it should be (40,1)
# check train_features[0][0] shape, (1,) -> it should be (1,1)

#print(train_features[0].shape)

#load results 
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "rb") as f:
    results = pkl.load(f)

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_name)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+'_'+MODEL+'_test_R2_plot.png', dpi=300, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_name)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+'_'+MODEL+'_train_R2_plot.png', dpi=300, bbox_inches='tight')


# Test with new function that has limited convergence to see if it impacts the test R2

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score

#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y[:, i].cpu().numpy()
            test_target = test_y[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv.pkl", "wb") as f:
    pkl.dump(results, f)

Epoch [1000/800000], Property: num_nodes, Loss: 11481.0527
Epoch [2000/800000], Property: num_nodes, Loss: 10354.6318
Epoch [3000/800000], Property: num_nodes, Loss: 9404.4902
Epoch [4000/800000], Property: num_nodes, Loss: 8578.0361
Epoch [5000/800000], Property: num_nodes, Loss: 7852.7349
Epoch [6000/800000], Property: num_nodes, Loss: 7213.9277
Epoch [7000/800000], Property: num_nodes, Loss: 6650.7715
Epoch [8000/800000], Property: num_nodes, Loss: 6155.5811
Epoch [9000/800000], Property: num_nodes, Loss: 5721.0806
Epoch [10000/800000], Property: num_nodes, Loss: 5339.2759
Epoch [11000/800000], Property: num_nodes, Loss: 5001.8472
Epoch [12000/800000], Property: num_nodes, Loss: 4702.5679
Epoch [13000/800000], Property: num_nodes, Loss: 4437.0811
Epoch [14000/800000], Property: num_nodes, Loss: 4201.8760
Epoch [15000/800000], Property: num_nodes, Loss: 3993.8069
Epoch [16000/800000], Property: num_nodes, Loss: 3809.1426
Epoch [17000/800000], Property: num_nodes, Loss: 3644.1562
Epoc

Epoch [14000/800000], Property: num_edges, Loss: 201989.3438
Epoch [15000/800000], Property: num_edges, Loss: 197815.3906
Epoch [16000/800000], Property: num_edges, Loss: 193774.9219
Epoch [17000/800000], Property: num_edges, Loss: 189860.9375
Epoch [18000/800000], Property: num_edges, Loss: 186067.3281
Epoch [19000/800000], Property: num_edges, Loss: 182390.3906
Epoch [20000/800000], Property: num_edges, Loss: 178827.5156
Epoch [21000/800000], Property: num_edges, Loss: 175376.9531
Epoch [22000/800000], Property: num_edges, Loss: 172037.3750
Epoch [23000/800000], Property: num_edges, Loss: 168805.4062
Epoch [24000/800000], Property: num_edges, Loss: 165675.8750
Epoch [25000/800000], Property: num_edges, Loss: 162644.0625
Epoch [26000/800000], Property: num_edges, Loss: 159705.7969
Epoch [27000/800000], Property: num_edges, Loss: 156858.9219
Epoch [28000/800000], Property: num_edges, Loss: 154101.3438
Epoch [29000/800000], Property: num_edges, Loss: 151430.4219
Epoch [30000/800000], Pr

Epoch [149000/800000], Property: num_edges, Loss: 55439.3281
Epoch [150000/800000], Property: num_edges, Loss: 55195.7188
Epoch [151000/800000], Property: num_edges, Loss: 54954.0586
Epoch [152000/800000], Property: num_edges, Loss: 54714.0273
Epoch [153000/800000], Property: num_edges, Loss: 54475.4180
Epoch [154000/800000], Property: num_edges, Loss: 54238.1094
Epoch [155000/800000], Property: num_edges, Loss: 54002.1641
Epoch [156000/800000], Property: num_edges, Loss: 53767.5547
Epoch [157000/800000], Property: num_edges, Loss: 53534.2891
Epoch [158000/800000], Property: num_edges, Loss: 53302.3359
Epoch [159000/800000], Property: num_edges, Loss: 53071.6445
Epoch [160000/800000], Property: num_edges, Loss: 52842.1680
Epoch [161000/800000], Property: num_edges, Loss: 52613.8086
Epoch [162000/800000], Property: num_edges, Loss: 52386.4375
Epoch [163000/800000], Property: num_edges, Loss: 52159.8906
Epoch [164000/800000], Property: num_edges, Loss: 51934.0234
Epoch [165000/800000], P

Epoch [284000/800000], Property: num_edges, Loss: 29427.6289
Epoch [285000/800000], Property: num_edges, Loss: 29277.3535
Epoch [286000/800000], Property: num_edges, Loss: 29127.7383
Epoch [287000/800000], Property: num_edges, Loss: 28978.7812
Epoch [288000/800000], Property: num_edges, Loss: 28830.4922
Epoch [289000/800000], Property: num_edges, Loss: 28682.8574
Epoch [290000/800000], Property: num_edges, Loss: 28535.8848
Epoch [291000/800000], Property: num_edges, Loss: 28389.5762
Epoch [292000/800000], Property: num_edges, Loss: 28243.9297
Epoch [293000/800000], Property: num_edges, Loss: 28098.9414
Epoch [294000/800000], Property: num_edges, Loss: 27954.6152
Epoch [295000/800000], Property: num_edges, Loss: 27810.9512
Epoch [296000/800000], Property: num_edges, Loss: 27667.9492
Epoch [297000/800000], Property: num_edges, Loss: 27525.6094
Epoch [298000/800000], Property: num_edges, Loss: 27383.9316
Epoch [299000/800000], Property: num_edges, Loss: 27242.9102
Epoch [300000/800000], P

Epoch [419000/800000], Property: num_edges, Loss: 15172.1641
Epoch [420000/800000], Property: num_edges, Loss: 15112.4795
Epoch [421000/800000], Property: num_edges, Loss: 15053.4180
Epoch [422000/800000], Property: num_edges, Loss: 14994.9795
Epoch [423000/800000], Property: num_edges, Loss: 14937.1631
Epoch [424000/800000], Property: num_edges, Loss: 14879.9668
Epoch [425000/800000], Property: num_edges, Loss: 14823.3945
Epoch [426000/800000], Property: num_edges, Loss: 14767.4434
Epoch [427000/800000], Property: num_edges, Loss: 14712.1133
Epoch [428000/800000], Property: num_edges, Loss: 14657.4053
Epoch [429000/800000], Property: num_edges, Loss: 14603.3213
Epoch [430000/800000], Property: num_edges, Loss: 14549.8545
Epoch [431000/800000], Property: num_edges, Loss: 14497.0137
Epoch [432000/800000], Property: num_edges, Loss: 14444.7920
Epoch [433000/800000], Property: num_edges, Loss: 14393.1953
Epoch [434000/800000], Property: num_edges, Loss: 14342.2178
Epoch [435000/800000], P

...

Epoch [582000/800000], Property: num_triangles, Loss: 535889.5000
Epoch [583000/800000], Property: num_triangles, Loss: 535050.6875
Epoch [584000/800000], Property: num_triangles, Loss: 534213.8125
Epoch [585000/800000], Property: num_triangles, Loss: 533378.9375
Epoch [586000/800000], Property: num_triangles, Loss: 532545.8750
Epoch [587000/800000], Property: num_triangles, Loss: 531714.8750
Epoch [588000/800000], Property: num_triangles, Loss: 530885.6875
Epoch [589000/800000], Property: num_triangles, Loss: 530058.4375
Epoch [590000/800000], Property: num_triangles, Loss: 529233.1250
Epoch [591000/800000], Property: num_triangles, Loss: 528409.8125
Epoch [592000/800000], Property: num_triangles, Loss: 527588.3750
Epoch [593000/800000], Property: num_triangles, Loss: 526768.8750
Epoch [594000/800000], Property: num_triangles, Loss: 525951.3125
Epoch [595000/800000], Property: num_triangles, Loss: 525135.6875
Epoch [596000/800000], Property: num_triangles, Loss: 524322.0000
Epoch [597

Epoch [707000/800000], Property: num_triangles, Loss: 445998.3438
Epoch [708000/800000], Property: num_triangles, Loss: 445400.7812
Epoch [709000/800000], Property: num_triangles, Loss: 444805.2500
Epoch [710000/800000], Property: num_triangles, Loss: 444211.5625
Epoch [711000/800000], Property: num_triangles, Loss: 443619.8438
Epoch [712000/800000], Property: num_triangles, Loss: 443030.0312
Epoch [713000/800000], Property: num_triangles, Loss: 442442.1250
Epoch [714000/800000], Property: num_triangles, Loss: 441856.2500
Epoch [715000/800000], Property: num_triangles, Loss: 441272.1875
Epoch [716000/800000], Property: num_triangles, Loss: 440690.1250
Epoch [717000/800000], Property: num_triangles, Loss: 440109.9375
Epoch [718000/800000], Property: num_triangles, Loss: 439531.7500
Epoch [719000/800000], Property: num_triangles, Loss: 438955.4062
Epoch [720000/800000], Property: num_triangles, Loss: 438381.0938
Epoch [721000/800000], Property: num_triangles, Loss: 437808.6250
Epoch [722

Epoch [9000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 10706.4639
Epoch [10000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 10501.5576
Epoch [11000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 10298.7305
Epoch [12000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 10097.9346
Epoch [13000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 9899.1162
Epoch [14000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 9702.2793
Epoch [15000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 9507.4258
Epoch [16000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 9314.5557
Epoch [17000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 9123.6699
Epoch [18000/800000], Property: number_of_n

Epoch [87000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 720.3068
Epoch [88000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 670.5511
Epoch [89000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 622.6504
Epoch [90000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 576.5867
Epoch [91000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 532.3412
Epoch [92000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 489.9462
Epoch [93000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 449.6524
Epoch [94000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 411.1507
Epoch [95000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 374.4417
Epoch [96000/800000], Property: number_of_nodes_in_the_

Epoch [165000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9684
Epoch [166000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9627
Epoch [167000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9593
Epoch [168000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9577
Epoch [169000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9571
Epoch [170000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9571
Epoch [171000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9571
Epoch [172000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9571
Epoch [173000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 11.9571
Early stopping at epoch 173231
Embedding: torch.Size([1

In [22]:
#load results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv.pkl", "rb") as f:
    results = pkl.load(f)

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_name)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+"_"+MODEL+'_test_R2_plot_limited_cv.png', dpi=300, bbox_inches='tight')

In [ ]:
# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_name)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+"_"+MODEL+'_train_R2_plot_limited_cv.png', dpi=300, bbox_inches='tight')

#### Test with more properties

In [25]:
import networkx as nx
import numpy as np

def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Use the average path length of the largest connected component for disconnected graphs
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    
def betweenness_centralization(G):
    n = len(G)
    betweenness = nx.betweenness_centrality(G)
    max_betweenness = max(betweenness.values())
    centralization = sum(max_betweenness - bet for bet in betweenness.values())
    if n > 2:
        centralization /= (n - 1) * (n - 2) / 2
    return centralization

def pagerank_centralization(G, alpha=0.85):
    n = len(G)
    pagerank = nx.pagerank(G, alpha=alpha)
    max_pagerank = max(pagerank.values())
    centralization = sum(max_pagerank - pr for pr in pagerank.values())
    if n > 1:
        centralization /= (n - 1)
    return centralization

def clustering_properties(G):
    average_clustering = nx.average_clustering(G)
    transitivity = nx.transitivity(G)
    return average_clustering, transitivity

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        
        # Number of nodes
        num_nodes = G.number_of_nodes()
        
        # Number of edges
        num_edges = G.number_of_edges()
        
        # Density
        density = nx.density(G)
        
        # Average Path Length
        avg_path_len = calculate_avg_path_length(G)
        
        # Diameter
        if nx.is_connected(G):
            diameter = nx.diameter(G)
        else:
            # Use the diameter of the largest connected component for disconnected graphs
            components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
            largest_component = max(components, key=len)
            diameter = nx.diameter(largest_component)
        
        # Radius
        if nx.is_connected(G):
            radius = nx.radius(G)
        else:
            radius = nx.radius(largest_component)
        
        # Clustering Coefficient
        clustering_coeff = nx.average_clustering(G)
        
        # Transitivity
        transitivity = nx.transitivity(G)
        
        # Assortativity
        assortativity = nx.degree_assortativity_coefficient(G)
        
        # Number of Cliques
        num_cliques = len(list(nx.find_cliques(G)))
        
        # Number of Triangles
        num_triangles = sum(nx.triangles(G).values()) / 3
        
        # Number of Squares (4-cycles)
        num_squares = sum(nx.square_clustering(G).values()) / 4
        
        # Size of the Largest Connected Component
        largest_component_size = len(max(nx.connected_components(G), key=len))
        
        # Average Degree
        degrees = [d for n, d in G.degree()]
        avg_degree = np.mean(degrees)
        
        # Betweenness Centrality
        betweenness_centrality = nx.betweenness_centrality(G)
        avg_betweenness_centrality = np.mean(list(betweenness_centrality.values()))
        
        # Eigenvalues of the Adjacency Matrix (for spectral properties)
        eigenvalues = np.linalg.eigvals(nx.adjacency_matrix(G).todense())
        spectral_radius = max(eigenvalues)
        algebraic_connectivity = sorted(eigenvalues)[1]  # second smallest eigenvalue
        
        # Graph Laplacian Eigenvalues
        laplacian_eigenvalues = np.linalg.eigvals(nx.laplacian_matrix(G).todense())
        graph_energy = sum(abs(laplacian_eigenvalues))
        
        # Small-World-ness
        # Compare clustering coefficient and average path length with those of a random graph
        random_graph = nx.gnm_random_graph(num_nodes, num_edges)
        random_clustering_coeff = nx.average_clustering(random_graph)
        random_avg_path_len = calculate_avg_path_length(random_graph)
        small_world_coefficient = (clustering_coeff / random_clustering_coeff) / (avg_path_len / random_avg_path_len)

        # Calculate Betweenness Centralization
        betweenness_cent = betweenness_centralization(G)
        print(f"Betweenness Centralization: {betweenness_cent}")

        # Calculate PageRank Centralization
        pagerank_cent = pagerank_centralization(G)
        print(f"PageRank Centralization: {pagerank_cent}")

        # Calculate Clustering properties
        avg_clustering, transitivity = clustering_properties(G)
        print(f"Average Clustering Coefficient: {avg_clustering}")
        print(f"Transitivity: {transitivity}")
        
        properties.append((
            num_nodes,
            num_edges,
            density,
            avg_path_len,
            diameter,
            radius,
            clustering_coeff,
            transitivity,
            assortativity,
            num_cliques,
            num_triangles,
            num_squares,
            largest_component_size,
            avg_degree,
            avg_betweenness_centrality,
            spectral_radius,
            algebraic_connectivity,
            graph_energy,
            small_world_coefficient
        ))
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties_long = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties_long = compute_graph_properties(selected_dataset)

#save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "wb") as f:
    pkl.dump(train_properties_long, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "wb") as f:
    pkl.dump(test_properties_long, f)



Betweenness Centralization: 0.001724022477030078
PageRank Centralization: 0.011393522233220854
Average Clustering Coefficient: 0.46590185426058695
Transitivity: 0.5531747591900924
Betweenness Centralization: 0.0017261983853566708
PageRank Centralization: 0.00892683265732043
Average Clustering Coefficient: 0.5182009816568728
Transitivity: 0.5797223390828776
Betweenness Centralization: 0.003333083015572392
PageRank Centralization: 0.015313849504207902
Average Clustering Coefficient: 0.4704590487485227
Transitivity: 0.4714022140221402
Betweenness Centralization: 0.0010671877568257148
PageRank Centralization: 0.0052237203197423076
Average Clustering Coefficient: 0.5604119583322708
Transitivity: 0.5464108308171812
Betweenness Centralization: 0.0022241433151757965
PageRank Centralization: 0.010110693464664746
Average Clustering Coefficient: 0.5817108880061148
Transitivity: 0.6164350158497928
Betweenness Centralization: 0.0008253715900658974
PageRank Centralization: 0.008937849000785933
Avera

Betweenness Centralization: 0.001078428822044226
PageRank Centralization: 0.009063561996750147
Average Clustering Coefficient: 0.5532543094005744
Transitivity: 0.5523364485981308
Betweenness Centralization: 0.001653614405354768
PageRank Centralization: 0.007032983882622251
Average Clustering Coefficient: 0.5508670653133515
Transitivity: 0.5519043035886154
Betweenness Centralization: 0.0015445362029709983
PageRank Centralization: 0.00937031329613198
Average Clustering Coefficient: 0.503380722715761
Transitivity: 0.5818673883626523
Betweenness Centralization: 0.0017807546976311142
PageRank Centralization: 0.009501326946814097
Average Clustering Coefficient: 0.4149566523606274
Transitivity: 0.4692404834926935
Betweenness Centralization: 0.0020229469275225478
PageRank Centralization: 0.007260008922385327
Average Clustering Coefficient: 0.5209656672192529
Transitivity: 0.5912332579579057
Betweenness Centralization: 0.0011742922774086507
PageRank Centralization: 0.0074486415980829805
Average

Betweenness Centralization: 0.0010914474082738801
PageRank Centralization: 0.009126440311914046
Average Clustering Coefficient: 0.4677095114948748
Transitivity: 0.47178947368421054
Betweenness Centralization: 0.0012685107404340795
PageRank Centralization: 0.006816839320975924
Average Clustering Coefficient: 0.5186194311514003
Transitivity: 0.5033734232912878
Betweenness Centralization: 0.0009491348124185677
PageRank Centralization: 0.006918374381046993
Average Clustering Coefficient: 0.48102698538622907
Transitivity: 0.4596774193548387
Betweenness Centralization: 0.0012768517676512569
PageRank Centralization: 0.012649930264661277
Average Clustering Coefficient: 0.5059288865619318
Transitivity: 0.4874008810572687
Betweenness Centralization: 0.0009213240690828585
PageRank Centralization: 0.01080048500110151
Average Clustering Coefficient: 0.4979806981681155
Transitivity: 0.5099398139704542
Betweenness Centralization: 0.002157265795254701
PageRank Centralization: 0.008179831460735679
Aver

Betweenness Centralization: 0.0015906975639688602
PageRank Centralization: 0.009703507119299034
Average Clustering Coefficient: 0.4380468922164223
Transitivity: 0.4976958525345622
Betweenness Centralization: 0.0018844204684231544
PageRank Centralization: 0.009696681730930105
Average Clustering Coefficient: 0.4778778278778279
Transitivity: 0.46482213438735176
Betweenness Centralization: 0.0014654430088957636
PageRank Centralization: 0.007175270605341495
Average Clustering Coefficient: 0.5313838022553865
Transitivity: 0.5791930643547849
Betweenness Centralization: 0.0023295450822815385
PageRank Centralization: 0.007339947681879867
Average Clustering Coefficient: 0.4882268866324595
Transitivity: 0.49248774315989247
Betweenness Centralization: 0.0014804367483372718
PageRank Centralization: 0.006898064630729408
Average Clustering Coefficient: 0.4274698304110067
Transitivity: 0.4977714748784441
Betweenness Centralization: 0.0013696492907756873
PageRank Centralization: 0.006859673356675676
Av

Betweenness Centralization: 0.0023663928569996754
PageRank Centralization: 0.010479937220992766
Average Clustering Coefficient: 0.40516212087419506
Transitivity: 0.4718985606579849
Betweenness Centralization: 0.0007162682787337847
PageRank Centralization: 0.008947032081459878
Average Clustering Coefficient: 0.41375953274526434
Transitivity: 0.4521642619311876
Betweenness Centralization: 0.0011434019801239968
PageRank Centralization: 0.009548973385956889
Average Clustering Coefficient: 0.41876977368463436
Transitivity: 0.4487510766580534
Betweenness Centralization: 0.0015447532700325556
PageRank Centralization: 0.01063250198665506
Average Clustering Coefficient: 0.46562118234926914
Transitivity: 0.4886090278760436
Betweenness Centralization: 0.0010981793433088153
PageRank Centralization: 0.010430659885190638
Average Clustering Coefficient: 0.49216030073268235
Transitivity: 0.5008333333333334
Betweenness Centralization: 0.0010822288739901177
PageRank Centralization: 0.009496451064002627


...

Betweenness Centralization: 0.0016015888608094807
PageRank Centralization: 0.008690623973229263
Average Clustering Coefficient: 0.5515680074517951
Transitivity: 0.5680586547656997
Betweenness Centralization: 0.0011152757709308823
PageRank Centralization: 0.006271841501676609
Average Clustering Coefficient: 0.48315944816959006
Transitivity: 0.49604933513201
Betweenness Centralization: 0.0014292724449325037
PageRank Centralization: 0.009932911572938054
Average Clustering Coefficient: 0.4742735744374238
Transitivity: 0.5309236947791165
Betweenness Centralization: 0.0014965941334372744
PageRank Centralization: 0.012584662774659255
Average Clustering Coefficient: 0.3914531938770166
Transitivity: 0.41455891425046265
Betweenness Centralization: 0.001208716667858161
PageRank Centralization: 0.011089412664039719
Average Clustering Coefficient: 0.4570852841497984
Transitivity: 0.512407172613657
Betweenness Centralization: 0.0017645748807944511
PageRank Centralization: 0.008612348345212266
Averag

Betweenness Centralization: 0.0016893666604143266
PageRank Centralization: 0.007261296426249009
Average Clustering Coefficient: 0.4895942907095146
Transitivity: 0.5292717753450737
Betweenness Centralization: 0.0018209978285594372
PageRank Centralization: 0.00897324572453724
Average Clustering Coefficient: 0.4776766954642951
Transitivity: 0.5410855635130042
Betweenness Centralization: 0.0012753723987881404
PageRank Centralization: 0.007274189747062118
Average Clustering Coefficient: 0.43347151119790084
Transitivity: 0.5371937639198219
Betweenness Centralization: 0.0018873677135838164
PageRank Centralization: 0.013900461549921276
Average Clustering Coefficient: 0.4474664119096523
Transitivity: 0.5376685170499603
Betweenness Centralization: 0.0015807739224130268
PageRank Centralization: 0.0075140645952897385
Average Clustering Coefficient: 0.43447924340507466
Transitivity: 0.45174129353233833
Betweenness Centralization: 0.002425460497996911
PageRank Centralization: 0.011985844008400536
Av

Betweenness Centralization: 0.0017082454844297328
PageRank Centralization: 0.0072574198146886995
Average Clustering Coefficient: 0.5042735176880524
Transitivity: 0.5379715825575698
Betweenness Centralization: 0.001079859727098108
PageRank Centralization: 0.007172021658469183
Average Clustering Coefficient: 0.5102042836349443
Transitivity: 0.48226221079691517
Betweenness Centralization: 0.0012215599777280182
PageRank Centralization: 0.009627955726328253
Average Clustering Coefficient: 0.4894410093462882
Transitivity: 0.5043808894032071
Betweenness Centralization: 0.0011317265369667152
PageRank Centralization: 0.009382661878394587
Average Clustering Coefficient: 0.5234762631904986
Transitivity: 0.5461741424802111
Betweenness Centralization: 0.000982121596426987
PageRank Centralization: 0.007434030500673798
Average Clustering Coefficient: 0.534659014193242
Transitivity: 0.5500625782227785
Betweenness Centralization: 0.001879980122457299
PageRank Centralization: 0.008932607457962578
Averag

Betweenness Centralization: 0.0020427746935174295
PageRank Centralization: 0.008514572099399582
Average Clustering Coefficient: 0.458322257664363
Transitivity: 0.5150893641957223
Betweenness Centralization: 0.0009497320032780242
PageRank Centralization: 0.0075399618318635125
Average Clustering Coefficient: 0.4868892026841561
Transitivity: 0.4892683691054363
Betweenness Centralization: 0.0017553668978511963
PageRank Centralization: 0.007050286215844919
Average Clustering Coefficient: 0.4287121164276337
Transitivity: 0.4963527851458886
Betweenness Centralization: 0.0008140083005664208
PageRank Centralization: 0.006302896899957454
Average Clustering Coefficient: 0.5162148268325981
Transitivity: 0.5791429279167699
Betweenness Centralization: 0.0012072696496697867
PageRank Centralization: 0.008905452454419848
Average Clustering Coefficient: 0.44398809720817833
Transitivity: 0.4521181001283697
Betweenness Centralization: 0.0013891680341095573
PageRank Centralization: 0.00581845930448916
Aver

Betweenness Centralization: 0.0015868546427589758
PageRank Centralization: 0.008883798900057973
Average Clustering Coefficient: 0.45325290131324597
Transitivity: 0.45181255526083114
Betweenness Centralization: 0.0023466568829991195
PageRank Centralization: 0.008319393588301272
Average Clustering Coefficient: 0.4301510682041654
Transitivity: 0.4356193142057383
Betweenness Centralization: 0.002862916203205466
PageRank Centralization: 0.013808503447373773
Average Clustering Coefficient: 0.26507936507936497
Transitivity: 0.35
Betweenness Centralization: 0.001929097615535771
PageRank Centralization: 0.010670642573089873
Average Clustering Coefficient: 0.5096106686635118
Transitivity: 0.5312923851966673
Betweenness Centralization: 0.0015849851714680877
PageRank Centralization: 0.006561513407075881
Average Clustering Coefficient: 0.5243943790018559
Transitivity: 0.536086051353227
Betweenness Centralization: 0.001016438056481488
PageRank Centralization: 0.008980251930315935
Average Clustering 

In [26]:
#load the properties
with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "rb") as f:
    train_properties_long = pkl.load(f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "rb") as f:
    test_properties_long = pkl.load(f)

In [27]:
#print the names of the properties
# print(train_properties_long[0].keys())

#print the first element of the properties
# print(train_properties_long[0])

#copare train_properties and train_properties_long
print(train_properties[0])
print(train_properties_long[0])
print(len(train_properties_long))

(116, 488, 0.07316341829085457, 3.314842578710645, 148, 938.0, 6.439699892510922, 116)
(116, 488, 0.07316341829085457, 3.314842578710645, 7, 4, 0.46590185426058695, 0.5531747591900924, 0.4167832357774365, 148, 938.0, 6.439699892510922, 116, 8.413793103448276, 0.02030563665535653, 14.955872318929243, -4.061650998933752, 976.0000000000015, 5.160209267107621)
1523


In [ ]:
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']
train_y_long = torch.tensor(train_properties_long, dtype=torch.float32)
test_y_long = torch.tensor(test_properties_long, dtype=torch.float32)
#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names_long):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y_long[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y_long[:, i].cpu().numpy()
            test_target = test_y_long[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv_long.pkl", "wb") as f:
    pkl.dump(results, f)

/scratch-local/tpelletreaudur.7095885/ipykernel_3365956/1310961172.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  train_y_long = torch.tensor(train_properties_long, dtype=torch.float32)
/scratch-local/tpelletreaudur.7095885/ipykernel_3365956/1310961172.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  test_y_long = torch.tensor(test_properties_long, dtype=torch.float32)


Epoch [1000/800000], Property: num_nodes, Loss: 11468.3633
Epoch [2000/800000], Property: num_nodes, Loss: 10342.7480
Epoch [3000/800000], Property: num_nodes, Loss: 9393.3330
Epoch [4000/800000], Property: num_nodes, Loss: 8567.7305
Epoch [5000/800000], Property: num_nodes, Loss: 7843.1118
Epoch [6000/800000], Property: num_nodes, Loss: 7204.7290
Epoch [7000/800000], Property: num_nodes, Loss: 6641.7861
Epoch [8000/800000], Property: num_nodes, Loss: 6146.7651
Epoch [9000/800000], Property: num_nodes, Loss: 5712.5869
Epoch [10000/800000], Property: num_nodes, Loss: 5331.2368
Epoch [11000/800000], Property: num_nodes, Loss: 4994.2720
Epoch [12000/800000], Property: num_nodes, Loss: 4695.3525
Epoch [13000/800000], Property: num_nodes, Loss: 4430.1709
Epoch [14000/800000], Property: num_nodes, Loss: 4195.2544
Epoch [15000/800000], Property: num_nodes, Loss: 3987.5378
Epoch [16000/800000], Property: num_nodes, Loss: 3803.3008
Epoch [17000/800000], Property: num_nodes, Loss: 3638.7441
Epoc

In [ ]:
#load results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv_long.pkl", "rb") as f:
    results = pkl.load(f)

In [ ]:
#print color_long length and property_names_long length
print(len(colors_long))
print(len(property_names_long))

In [ ]:
property_names_long

In [ ]:
#import plt
import matplotlib.pyplot as plt
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']
#make a color list for the properties names but with different colors
colors_long = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'gray', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'lime', 'teal', 'lavender', 'tan']

plt.figure(figsize=(12, 8))

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_names_long)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+"_"+MODEL+'_test_R2_plot_limited_cv_long.png', dpi=300, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12, 8))
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']
colors_long = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'gray', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'lime', 'teal', 'lavender', 'tan']

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_names_long)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)

    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+"_"+MODEL+'_train_R2_plot_limited_cv_long.png', dpi=300, bbox_inches='tight')
    